In [1]:
#!nvidia-smi

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchsummary import summary

from DatasetLogReturn import StockDatasetLogReturn
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

import math
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt

In [3]:
configs = {
    "batch_size" : 128,
    "max_file_count" : 2179,
    "train_file" : 5,
    "validation_file" : 2,
    "test_file" : 2,
    "input_size" : 8,
    "output_size" : 1,
    "init_learning_rate" : 3e-4,
    "scheduler_factor": 0.8,
    "scheduler_patience": 3,
    "scheduler_min_lr": 2e-5,
    "epoch" : 30,
    "model_name" : "LSTM4",
}

configs["file_count"] = configs["train_file"] + configs["validation_file"] + configs["test_file"]
configs["model_folder"] = f"./models/{configs["model_name"]}/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
tickers = list(map(str.strip,open("tickers_sp500.txt","r").readlines()))
filenames = ["./data/" + x + ".csv" for x in tickers]
rd.shuffle(filenames)
filenames = filenames[:configs["file_count"]]

train_files, test_files = train_test_split(filenames,test_size=configs["test_file"]/configs["file_count"])
train_files, val_files = train_test_split(train_files,test_size=configs["validation_file"]/(configs["train_file"] + configs["validation_file"]))

train_dataset = StockDatasetLogReturn(train_files, configs["input_size"], configs["output_size"])
train_dataset.Preprocess()
train_loader = DataLoader(train_dataset,batch_size=configs["batch_size"],shuffle=True)

val_dataset = StockDatasetLogReturn(val_files,configs["input_size"],configs["output_size"])
val_dataset.Preprocess()
val_loader = DataLoader(val_dataset,batch_size=configs["batch_size"],shuffle=True)

test_dataset = StockDatasetLogReturn(test_files,configs["input_size"],configs["output_size"])
test_dataset.Preprocess()
test_loader = DataLoader(test_dataset,batch_size=configs["batch_size"],shuffle=True)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/26654 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9378 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/15806 [00:00<?, ?it/s]

In [4]:
torch.set_default_dtype(torch.float32)
class StockLSTM(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.hidden_size = 512
        self.batch_size = configs["batch_size"]
        self.layer1 = nn.LSTM(1,self.hidden_size,2,batch_first=True,bias=False)
        self.layer2 = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.LayerNorm(self.hidden_size),
            nn.Linear(self.hidden_size,1,bias=False),
        )
        self.layer3 = nn.Linear(1,1,bias=False)

    def forward(self, input) :
        input = torch.multiply(input,25)
        h1,(hn,cn) = self.layer1(input)
        output = self.layer2(h1[:,-1,:])
        output = torch.multiply(output,0.04)
        # output = self.layer3(h2)
        return output


In [36]:
model = StockLSTM()
model = model.to(device)

summary(model,input_size=(configs["batch_size"],configs["input_size"],1))

Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              3,147,776
├─Sequential: 1-2                        --
|    └─Dropout: 2-1                      --
|    └─LayerNorm: 2-2                    1,024
|    └─Linear: 2-3                       512
├─Linear: 1-3                            1
Total params: 3,149,313
Trainable params: 3,149,313
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              3,147,776
├─Sequential: 1-2                        --
|    └─Dropout: 2-1                      --
|    └─LayerNorm: 2-2                    1,024
|    └─Linear: 2-3                       512
├─Linear: 1-3                            1
Total params: 3,149,313
Trainable params: 3,149,313
Non-trainable params: 0

In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr = configs["init_learning_rate"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    'min',
    factor=configs['scheduler_factor'],
    patience=configs['scheduler_patience'],
    min_lr=configs['scheduler_min_lr']
)
loss_fn = torch.nn.MSELoss()

In [38]:
train_losses = []
val_losses = []
avarage_train_losses = []
avarage_val_losses = []
learning_rates = []

model_folder = configs["model_folder"]
best_loss = 1000000

for epoch in tqdm(range(configs["epoch"])) :

    model.train()

    learning_rates.append(optimizer.param_groups[0]['lr'])

    cur_train_loss = []
    for input,target in tqdm(train_loader) :

        close_price_input = input[:,4,:,None]
        close_price_target = target[:,4,:]
        close_price_input = close_price_input.to(device)
        close_price_target = close_price_target.to(device)

        if(close_price_input.shape[0] != configs["batch_size"]) :
            continue

        optimizer.zero_grad()

        output = model(close_price_input)

        loss = loss_fn(output,close_price_target)

        loss.backward()

        optimizer.step()

        train_losses.append(loss.to(torch.device("cpu")).item())
        cur_train_loss.append(loss.to(torch.device("cpu")).item())

    avarage_train_loss = np.mean(np.array(cur_train_loss))
    avarage_train_losses.append(avarage_train_loss)

    model.eval()

    with torch.no_grad() :
        cur_val_loss = []
        for input,target in tqdm(val_loader) :

            close_price_input = input[:,4,:,None]
            close_price_target = target[:,4,:]
            close_price_input = close_price_input.to(device)
            close_price_target = close_price_target.to(device)

            if(close_price_input.shape[0] != configs["batch_size"]) :
                continue

            output = model(close_price_input)

            loss = loss_fn(output,close_price_target)

            val_losses.append(loss.to(torch.device("cpu")).item())
            cur_val_loss.append(loss.to(torch.device("cpu")).item())

        avarage_val_loss = np.mean(np.array(cur_val_loss))
        avarage_val_losses.append(avarage_val_loss)

        scheduler.step(avarage_val_loss)
        
        if(best_loss > avarage_val_loss) :

            state = {
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),
                'train_loss': avarage_train_loss,
                'val_loss': avarage_val_loss,
                'best_val_loss': best_loss,
            }

            best_loss = avarage_val_loss

            torch.save(state, model_folder + "LSTM.pth.tar")

    print("train loss : ",avarage_train_loss)
    print("val loss : ",avarage_val_loss)


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/209 [00:00<?, ?it/s]

-- -7.344266487052664e-05
++ 5.548770786845125e-06
-- 6.358729297062382e-05
++ -0.004015755839645863
-- -0.0004656218516174704
++ 0.007376252673566341
-- -0.00010755291441455483
++ 0.004061204846948385
-- -5.4067742894403636e-05
++ -0.0011306452797725797
-- -2.7598685846896842e-05
++ -0.0003036178241018206
-- 0.0002723602228797972
++ -0.01055202353745699
-- -0.00011583044397411868
++ 0.0028039251919835806
-- 0.00011240242747589946
++ 0.001102498616091907
-- 5.794247772428207e-05
++ -8.700119360582903e-05
-- -0.00012361019616946578
++ 0.0008254335843957961
-- 0.0
++ 0.002511190716177225
-- -0.00010896877938648686
++ 0.007619414012879133
-- 0.00043861192534677684
++ 0.001150738913565874
-- -0.00017760256014298648
++ -0.0061962115578353405
-- 0.0
++ 0.0008498596725985408
-- 0.00017967497115023434
++ -0.0038211080245673656
-- -0.00045519295963458717
++ 0.010848534293472767
-- 0.0007706615724600852
++ 0.004166176076978445
-- 5.432568286778405e-05
++ 0.0002355367032578215
-- 1.01439854915952

  0%|          | 0/74 [00:00<?, ?it/s]

-- 0.0004419750184752047
++ 9.490639058640227e-05
-- 0.0
++ -0.0012109982781112194
-- -0.003511653281748295
++ -0.004431921523064375
-- -0.00011749719851650298
++ -4.5221597247291356e-05
-- -0.010547438636422157
++ 0.007327991537749767
-- 0.0
++ -0.00031582711380906403
-- 0.00039201873005367815
++ 0.0005411843303591013
-- 0.0
++ 0.008607217110693455
-- -0.003511789022013545
++ 8.068367606028914e-05
-- 8.390291623072699e-05
++ 0.0005759514751844108
-- -0.0036020756233483553
++ 0.020275834947824478
-- 0.00021935834956821054
++ -0.0009719878435134888
-- -4.8464298743056133e-05
++ -0.001350971288047731
-- 0.0
++ 0.0002926538872998208
-- 0.0
++ 0.0010906277457252145
-- -0.0002677063748706132
++ -0.0030070932116359472
-- -0.00017497323278803378
++ 0.0021199670154601336
-- 0.0010565328411757946
++ -0.00023036643688101321
-- -0.0005406326381489635
++ -0.0017652374226599932
-- 0.0
++ -0.0056753759272396564
-- 0.0017972382484003901
++ -0.0022709788754582405
-- -0.005809050519019365
++ -0.0038984

...

-- 0.0
++ 0.016477109864354134
-- 0.0
++ -0.004472915083169937
-- 4.9906166168511845e-06
++ -0.0017729264218360186
-- 9.75958118942799e-06
++ 0.0001829230022849515
-- 2.305283004488956e-07
++ -0.0002399963850621134
-- 2.5467770115028543e-07
++ -1.2063849680998828e-05
-- 7.397517038043588e-05
++ 0.0009500532178208232
-- 2.8925517199240858e-06
++ -0.000300667918054387
-- 1.8991913464105892e-07
++ 0.00042531226063147187
-- -2.625825459290354e-07
++ 0.0002179252333007753
-- -4.103108039998915e-07
++ -0.0001705815375316888
-- 1.3555782061303034e-06
++ 0.00021136541909072548
-- 3.6245135106582893e-06
++ -0.0006425322499126196
-- 3.0917435651645064e-05
++ -0.0017690842505544424
-- 2.2620512027060613e-05
++ -0.001008220249786973
-- 4.754941073770169e-06
++ 0.0005085369921289384
-- 3.201842218913953e-06
++ -0.0012166507076472044
-- 0.0
++ -0.0003378960827831179
-- 4.2279629269614816e-05
++ -0.0006905415211804211
-- 0.0
++ -0.0008626062190160155
-- 8.770662134338636e-06
++ 0.00011318981705699116

  0%|          | 0/209 [00:00<?, ?it/s]

-- 1.479051661590347e-06
++ 0.00010435752483317629
-- 3.0382971090148203e-06
++ -8.1002785009332e-05
-- 0.00017609789210837334
++ 0.0004297918058000505
-- -7.058125675030169e-07
++ 0.0001473752490710467
-- -1.0250049626847613e-06
++ -0.0009877149714156985
-- 4.279306813259609e-05
++ 0.0002252895646961406
-- 0.0
++ 6.902074528625235e-05
-- 2.6690513550420292e-05
++ -0.0003625304962042719
-- -1.5892895817160024e-06
++ 9.575856529409066e-05
-- -1.6465489807160338e-06
++ -1.9406985302339308e-05
-- 4.3064601413789205e-06
++ 0.0006795102963224053
-- 1.5828624100322486e-06
++ -0.0005841288366355002
-- 1.8251861320095486e-06
++ 0.0001841962366597727
-- 1.6753834643168375e-05
++ 0.0005322763463482261
-- 2.5671584808151238e-05
++ 0.0006553248385898769
-- 9.180562301480677e-06
++ 0.0001824522769311443
-- 3.6549288324749796e-06
++ 2.301980202901177e-05
-- -6.121673550296691e-07
++ 0.00014566676691174507
-- 0.0
++ -0.0006214143359102309
-- 3.5065490919805598e-06
++ 0.0005334613379091024
-- 3.632272

  0%|          | 0/74 [00:00<?, ?it/s]

-- -7.435519819409819e-06
++ 0.0002220743481302634
-- 0.0
++ 9.41272810450755e-05
-- 2.7689649868989363e-05
++ 0.00026461758534424007
-- 1.2841001080232672e-05
++ 0.00031852954998612404
-- -2.5923414796125144e-05
++ 0.003209527349099517
-- 2.6497425551497145e-06
++ -0.00029607556643895805
-- -2.6666670237318613e-05
++ -0.00078534334897995
-- -1.734008765197359e-05
++ 0.00031955158920027316
-- 1.402910584147321e-05
++ -0.0012551310937851667
-- -7.115519565559225e-06
++ 0.0017730244435369968
-- 0.0
++ 0.0015970313688740134
-- 0.0
++ -0.0004942517844028771
-- -1.2483705177146476e-05
++ 0.0007848397945053875
-- 0.0012039074208587408
++ -0.023862475529313087
-- 4.5722961658611894e-05
++ 0.00036760978400707245
-- -2.604599103506189e-05
++ -0.00027145451167598367
-- 1.875764428405091e-05
++ 0.00034408518695272505
-- 0.0
++ 0.0035296822898089886
-- 1.927211087604519e-05
++ 0.000213578314287588
-- 2.5284891307819635e-05
++ -0.00046365088201127946
-- -1.3504760318028275e-05
++ -0.000367524160537

In [5]:
plt.plot(avarage_train_losses)
plt.show()
plt.plot(avarage_val_losses)
plt.show()

NameError: name 'avarage_train_losses' is not defined

In [6]:
train_losses_file = open(configs["model_folder"] + "train_losses.txt",'w')
val_losses_file = open(configs["model_folder"] + "val_losses.txt","w")

for value in train_losses :
    train_losses_file.write(f"{value}\n")

for value in val_losses :
    val_losses_file.write(f"{value}\n")

train_losses_file.close()
val_losses_file.close()

NameError: name 'train_losses' is not defined

In [5]:
best_model = StockLSTM()
best_model.load_state_dict(torch.load(configs["model_folder"] + "LSTM.pth.tar")['model'])

best_model.to(device)
best_model.eval()

StockLSTM(
  (layer1): LSTM(1, 512, num_layers=2, bias=False, batch_first=True)
  (layer2): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=512, out_features=1, bias=False)
  )
  (layer3): Linear(in_features=1, out_features=1, bias=False)
)

In [6]:
with torch.no_grad() :
    cur_test_loss = []
    for input,target in tqdm(test_loader) :

        close_price_input = input[:,4,:,None]
        close_price_target = target[:,4,:]
        close_price_input = close_price_input.to(device)
        close_price_target = close_price_target.to(device)

        if(close_price_input.shape[0] != configs["batch_size"]) :
            continue

        output = best_model(close_price_input)

        loss = loss_fn(output,close_price_target)

        cur_test_loss.append(loss.to(torch.device("cpu")).item())

    avarage_test_loss = np.mean(np.array(cur_test_loss))

print(avarage_test_loss)

NameError: name 'test_loader' is not defined

In [6]:
def get_log_stock_prediction(model, stock_files) :
    model.eval()
    data_set = StockDatasetLogReturn(stock_files, configs["input_size"], 1)
    data_set.Preprocess()
    data_loader = DataLoader(data_set,configs["batch_size"])
    predict_res = []
    target_res = []
    
    with torch.no_grad() : 
        
        for input,target in tqdm(data_loader) :
            close_price_input = input[:,4,:,None]
            close_price_target = target[:,4,:]
            close_price_input = close_price_input.to(device)

            output = model(close_price_input)

            close_price_input = close_price_input.to(torch.device("cpu"))
            
            predict_res += output.to(torch.device("cpu")).reshape(-1).tolist()
            target_res += close_price_target.to(torch.device("cpu")).reshape(-1).tolist()

    return np.array(predict_res),np.array(target_res)


In [7]:
def trade_test(predict, real) :
    buy_pos = predict > 0
    total_profit = np.exp(np.sum(buy_pos * real))
    return total_profit

In [8]:
def print_trade(best_model, stocks, output_frame) :
    for stock in tqdm(stocks) :
        predict,target = get_log_stock_prediction(best_model, [stock])
        buy_and_hold_profit = trade_test(np.ones(target.shape),target)
        naive_forecast_profit = trade_test(np.concatenate((np.array([0]),(target > 0)))[:-1],target)
        random_profit = trade_test(np.random.randint(low=0,high=2,size=target.shape),target)
        total_profit = trade_test(predict,target)
        inverse_total_profit = trade_test(-predict,target)
        
        new_row = {
            "Name": stock,
            "Buy_and_Hold" : buy_and_hold_profit,
            "Naive_Forecast" : naive_forecast_profit,
            "Random_Profit" : random_profit,
            "Total_Profit" : total_profit,
            "Inverse_Total_Profit" : inverse_total_profit
        }
        
        output_frame.loc[len(output_frame.index)] = new_row

In [9]:
testTickers = list(map(str.strip,open("testTickers.txt","r").readlines()))
test_filenames = ["./data/" + x + ".csv" for x in testTickers]
output_frame = pd.DataFrame(columns=["Name","Buy_and_Hold","Naive_Forecast","Random_Profit","Total_Profit","Inverse_Total_Profit"])
print_trade(best_model,test_filenames,output_frame)
output_frame.to_csv("./out/" + configs["model_name"] + "out.csv",index = False)

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/24177 [00:00<?, ?it/s]

d:\Programming\Pattern_Reg_Final\DatasetLogReturn.py:48: RuntimeWarning: divide by zero encountered in log
  c = np.log(b)


  0%|          | 0/189 [00:00<?, ?it/s]

...

  0%|          | 0/5706 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/8217 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

In [15]:
plt.plot(learning_rates)
plt.show()

NameError: name 'learning_rates' is not defined